In [192]:
# Importing required libraries
import glob
import nltk
from nltk.stem import PorterStemmer
import os
# import binarytree as tree
from queue import LifoQueue 
import numpy as np

In [193]:
# Reading the stop word file
fObj = open('Stopword-List.txt', 'r')
SwContent = fObj.readlines()
swlist = [x.replace("\n","").replace(" ","") for x in SwContent]
swl = [x for x in swlist if x!=""]
fObj.close()

In [194]:
ldict = {}
ldict1 = {}

# Defining the path where the dataset files are stored
path = 'Dataset/*'

file_list = glob.glob(path)

porter = PorterStemmer()

# Looping over all files in the file list
for file in file_list:
    f = open(file, 'r')
    
    # Reading the entire file and removing punctuations
    full_file = f.read().replace(".", "").replace("n't", " not").replace("'", "").replace("]", " ").replace("[", "").replace(",", " ").replace("?", "").replace("\n", " ").replace("-", " ").split() 

    lower_case_doc = [porter.stem(x.lower()) for x in full_file]
    
    # Removing stop words
    without_stop_list = [x for x in lower_case_doc if x not in swl]
    
    # Trimming the file name and removing redundant '.txt'
    file_name = os.path.basename(file)
    file_name = file_name.split('.')[0]
    
    # Storing the file content in the dictionary
    ldict[file_name] = lower_case_doc
    ldict1[file_name] = without_stop_list
    
    f.close()
    
# Printing the dictionaries containing file contents
# print("ldict: ", ldict)
# print("ldict1: ", ldict1)

In [195]:
print(ldict.keys())

# temp = 'Speeches\\speech_0.txt'
# p = os.path.basename(temp)
# print(p.split('.')[0])

dict_keys(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5', '6', '7', '8', '9'])


In [196]:
# Initializing an empty dictionary to store the inverted index table
index_table = {}

# Looping over all keys in the ldict1 dictionary
for key in ldict1.keys():
    # Looping over all unique words in the key's value
    for word in set(ldict1[key]):
        if word not in index_table:
            index_table[word] = []
            index_table[word].append(key)
        else:
            index_table[word].append(key)

In [197]:
# Looping over all keys in the index table dictionary
for x in index_table.keys():
    # Using numpy to find the unique elements and their respective counts in the value list of the current key
    unique_elements, counts_elements = np.unique(index_table[x], return_counts=True)
    
    # Checking if the maximum count of any key in the value list is greater than 1
    if counts_elements.max() > 1:
        # If the maximum count is greater than 1, printing the current key
        print(x)


In [198]:
#positional index creation

pindex_table={}
for key in ldict.keys():
    count=0
    for word in ldict[key]:
        count+=1
        if word in swl:  # entertaining the presence of stop words in the file (increment index without doing anything)
            continue
        if word not in pindex_table:
            pindex_table[word]={}
            pindex_table[word][key]=[]
            pindex_table[word][key].append(count)
        else:
            if key not in pindex_table[word]:
                pindex_table[word][key]=[]
            pindex_table[word][key].append(count)

# print(pindex_table)

In [199]:
words = pindex_table.keys()
# print(words)

In [200]:
#proximity query function

def ProximityQueryProcessing(w1, w2, ProximityValue, pindex_table):
    word1={}
    word2={}
    for word in pindex_table.keys():
        if word == w1:
            for docid in pindex_table[word]:
                word1[docid]=pindex_table[word][docid]
        elif word==w2:
            for docid in pindex_table[word]:
                word2[docid]=pindex_table[word][docid]
                
    s1=set(word1.keys())
    s2=set(word2.keys())
    commonDoc=None
    commonDoc=s1.intersection(s2)
    
    print("s1 : ",s1,"\n\ns2 : ",s2)
    print("\n\n## Common in : ",commonDoc,"\n")
    listOfTrueDoc=[]
    if commonDoc is not None:
        for commonDoc in commonDoc:
                x=checkPositions(word1[commonDoc],word2[commonDoc], ProximityValue,commonDoc)
                if len(x)>0:
                    listOfTrueDoc.append(commonDoc)
    else:
        print("result not found")
        
    print("no of doc in result : ",len(listOfTrueDoc),"\nDocs -->",listOfTrueDoc)
    print("ProximityQueryProcessing")

In [201]:
# def phrasalIntersect(w1,w2,s):
#     result=[]
#     for pos1 in w1:
#         for pos2 in w2:
#             if abs(pos1-pos2)==1:
#                 result.append(s)
#             else:
#                 pass
#     return result

In [202]:
def Boolean_query_processing(stemmedQuery,index_table):
#     postfixedquery=listInPostfix(stemmedQuery)
    alldoclist=set(ldict.keys())    
    
#     for multiple NOT Query

    if 'and' not in stemmedQuery and 'or' not in stemmedQuery:
        countNot=[c for c in stemmedQuery if c=='not']
        if len(countNot)%2 == 0:
             for word in index_table.keys():
                    if word == stemmedQuery[len(countNot)]:
                        print(index_table[word])
        else:
            for word in index_table.keys():
                    if word == stemmedQuery[len(countNot)]:
                        foundlist=set(index_table[word])
                        nfList=alldoclist.difference(foundlist)
                        print("No of result docs :",len(nfList),"\nDocs -->",nfList)

                        
#     for multiple OR Query 

    elif 'and' not in stemmedQuery and 'not' not in stemmedQuery:
        noOR=[c for c in stemmedQuery if c=='or'] 
        if len(noOR)==1:
            s1=wordFoundList(stemmedQuery[0],index_table)
            s2=wordFoundList(stemmedQuery[2],index_table)

            s=s1.union(s2)
            print("No of result docs :",len(s),"\nDoc -->",s)

        elif len(noOR)==2:
            s1=wordFoundList(stemmedQuery[0],index_table)
            s2=wordFoundList(stemmedQuery[2],index_table)
            s3=wordFoundList(stemmedQuery[4],index_table)
            s=s3.union(s1.union(s2))
            print("No of result docs :",len(s),"\nDoc -->",s)

        elif len(noOR)==3:
            s1=wordFoundList(stemmedQuery[0],index_table)
            s2=wordFoundList(stemmedQuery[2],index_table)
            s3=wordFoundList(stemmedQuery[4],index_table)
            s4=wordFoundList(stemmedQuery[8],index_table)
            sf1=s4.union(s3.union(s1.union(s2)))
            print("No of result docs :",len(s),"\nDocs -->",s)

            
#     for multiple AND Query 
            
    elif 'or' not in stemmedQuery and 'not' not in stemmedQuery:
        noAnd=[c for c in stemmedQuery if c=='and']

        if len(noAnd)==1:
            s1=wordFoundList(stemmedQuery[0],index_table)
            s2=wordFoundList(stemmedQuery[2],index_table)
            s=s1.intersection(s2)
            print("No of result docs :",len(s),"\nDoc -->",s)

        elif len(noAnd)==2:
            s1=wordFoundList(stemmedQuery[0],index_table)
            s2=wordFoundList(stemmedQuery[2],index_table)
            s3=wordFoundList(stemmedQuery[4],index_table)
            s=s3.intersection(s1.intersection(s2))
            print("No of result docs :",len(s),"\nDocs -->",s)

        elif len(noAnd)==3:
            s1=wordFoundList(stemmedQuery[0],index_table)
            s2=wordFoundList(stemmedQuery[2],index_table)
            s3=wordFoundList(stemmedQuery[4],index_table)
            s4=wordFoundList(stemmedQuery[8],index_table)
            sf1=s4.intersection(s3.intersection(s1.intersection(s2)))
            print("No of result docs :",len(s),"Docs -->",s)
    else:
        print("it's a complex query")

In [203]:
#one word to three word phrasal query processing

def phrasalQueryProcessing(Query,pindex_table):
    if len(Query)==1:
        for word in pindex_table.keys():
            if word == Query[0]: 
                for docid in pindex_table[word]:
                    print(docid,"-->",pindex_table[word][docid])
    elif len(Query)==2 or len(Query)==3:
        w1={}
        w2={}
        w3={}
        for word in pindex_table.keys():
            if word == Query[0]:
                for docid in pindex_table[word]:
                    w1[docid]=pindex_table[word][docid]
            if word == Query[1]:
                for docid in pindex_table[word]:
                    w2[docid]=pindex_table[word][docid]
            if len(Query)==3:
                if word == Query[2]:
                    for docid in pindex_table[word]:
                        w3[docid]=pindex_table[word][docid]

        s1=set(w1.keys())
        s2=set(w2.keys())
        s=s1.intersection(s2)
        myList=[]
        if len(Query)==2:
            if s is not None:
                for s in s:
                    x=checkPositions(w1[s], w2[s], 0,s)
                    if len(x)>0:
                        myList.append(s)
        elif len(Query)==3:
            myList1=[]
            myList2=[]
            s3=set(w3.keys())
            print(s3)
            sl3=s.intersection(s3)
            print("sl3 --> ",sl3)
            if sl3 is not None:
                for sl3 in sl3:
                    x=checkPositions(w1[sl3], w2[sl3], 0,sl3) 
                    if len(x)>0:
                        myList1.append(sl3)
                    y=checkPositions(w2[sl3], w3[sl3], 0,sl3) 
                    if len(y)>0:
                        myList2.append(sl3)
                myList=[x for x in myList1 if x in myList2]
                    
        print("no of docs : ",len(myList),"\nDocs -->",myList)
        print("phrasalQueryProcessing")


In [206]:
query=input("Enter the query ")

# removing punctuations from query
query=query.replace(".","").replace("n't"," not").replace("]"," ").replace("["," ").replace(","," ").replace("?","").replace("/"," / ").split()

#stemming the query
stemmedQuery=[porter.stem(x.lower()) for x in query]

# print(stemmedQuery)
Boolean_query_processing(stemmedQuery, index_table)

QueryWithoutStoplist = [x for x in stemmedQuery if x not in swl]
# print(QueryWithoutStoplist)


Enter the query good AND chase
No of result docs : 5 
Doc --> {'28', '30', '2', '27', '16'}


In [185]:
#query identification part

ProximityValue=0

#identifying the proximity query
if '/' in stemmedQuery:
    ProximityValue = int(QueryWithoutStoplist[QueryWithoutStoplist.index('/')+1])
    w1=QueryWithoutStoplist[0]
    w2=QueryWithoutStoplist[1]
    if ProximityValue != 0:
        ProximityQueryProcessing(w1,w2,ProximityValue,pindex_table)
    else:
        print("invalid proximity query")
        
#identifying the phrasal query        
elif 'not' not in stemmedQuery and 'and' not in stemmedQuery and 'or' not in stemmedQuery:
    if len(stemmedQuery)!=0:
        phrasalQueryProcessing(stemmedQuery,pindex_table)
    else:
        print("the query is null")
    
#identifying the boolean query
else:
    if 'and' in stemmedQuery or 'or' in stemmedQuery or 'not' in stemmedQuery: 
        Boolean_query_processing(stemmedQuery,index_table)
# elif 'not','and','or' in stemmedQuery:
#      print("its a boolean query")    


No of result docs : 10 
Doc --> {'3', '18', '2', '27', '22', '19', '9', '25', '16', '23'}


In [186]:
#finding list of docs where word has occurred

def wordFoundList(w,index_table):
#     print("in word found func ")

    for word in index_table.keys():
        if word == w:
            return set(index_table[word])

In [187]:
#post fix list to insert into parsing tree for complex queries

def listInPostfix(query):
    print("convert to posfix list")
    prec = {}
    prec['not']=3
    prec['or']=2
    prec['and']=1
    pfquery=[]
    myStack=LifoQueue()
    for q in query:
        if q not in 'and' or 'or' or 'not' or '(' or ')':
            pfquery.append(q)
        elif q=='(':
            myStack.push(q)
        elif q==')':
            topval=myStack.pop()
            while topval!='(':
                pfquery.append(topval)
                topval=myStack.pop()
        else:
            while (myStack.qsize()!=0) and (prec[myStack.top()]>=prec[q]):
                 pfquery.append(myStack.top())
            myStack.push(q)
                                             
    while myStack.qsize() !=0:
        pfquery.append(myStack.pop())  
                                             
    return pfquery

In [188]:
# Define a function to check the proximity of two words within a specified distance in a given document
def checkPositions(word1, word2, ProximityValue, commonDoc): 
    result = []
    for pos1 in word1:
        for pos2 in word2:
            # Check if the absolute difference between the positions of word1 and word2 is equal to ProximityValue
            if (abs(pos1-pos2)-1) == ProximityValue:
                # If the condition is satisfied, print the document ID, positions of word1 and word2, and the proximity value
                print("DOC--> ",commonDoc,"-->pos1(",pos1,") - pos2(",pos2,")",abs(pos1-pos2)-1)
                result.append(commonDoc)
            else:
                # If the condition is not satisfied, print a false statement with the document ID, positions of word1 and word2, and the proximity value
                pass
                print("false statement : \n","DOC--> ",commonDoc,"-->pos1(",pos1,") - pos2(",pos2,")",abs(pos1-pos2)-1)
    # Return the result list
    return result
